# 开胃小菜

In [1]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
group = [6,5,7]
values = [9,8,7,6,5,4,3,2,1]
sort_priority(values, group)

values

[5, 6, 7, 1, 2, 3, 4, 8, 9]

# 什么是闭包？闭包有什么用？为什么要用闭包？
我们就带着这3个问题来一步一步认识闭包。

In [30]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"

    def printer():
        # printer是嵌套函数
        print(msg)
    printer()
# 输出 zen of python
print_msg()

对于嵌套函数，它可以访问到其外层作用域中声明的非局部（non-local）变量，比如代码示例中的变量 msg 可以被嵌套函数 printer 正常访问。

那么有没有一种可能即使脱离了函数本身的作用范围，局部变量还可以被访问得到呢？答案是闭包

# 什么是闭包

函数身为第一类对象，它可以作为函数的返回值返回，现在我们来考虑如下的例子：

In [1]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"
    def printer():
        # printer 是嵌套函数
        print(msg)
    return printer

another = print_msg()
# 输出 zen of python
another()

zen of python


这段代码和前面例子的效果完全一样，同样输出 "zen of python"。不同的地方在于内部函数 **printer 直接作为返回值返回了。**

一般情况下，函数中的局部变量仅在函数的执行期间可用，一旦 print_msg() 执行过后，我们会认为 msg变量将不再可用。然而，在这里我们发现 print_msg 执行完之后，在调用 another 的时候 msg 变量的值正常输出了，这就是闭包的作用，闭包使得局部变量在函数外被访问成为可能。

这里的 another 就是一个闭包，闭包本质上是一个函数，它有两部分组成，printer 函数和变量 msg。闭包使得这些变量的值始终保存在内存中。

闭包，顾名思义，就是一个封闭的包裹，里面包裹着自由变量，就像在类里面定义的属性值一样，自由变量的可见范围随同包裹，哪里可以访问到这个包裹，哪里就可以访问到这个自由变量。

# 为什么要使用闭包

闭包避免了使用全局变量，此外，闭包允许将函数与其所操作的某些数据（环境）关连起来。这一点与面向对象编程是非常类似的，在面对象编程中，对象允许我们将某些数据（对象的属性）与一个或者多个方法相关联。

一般来说，当对象中只有一个方法时，这时使用闭包是更好的选择。来看一个例子：

In [4]:
def adder(x):
    def wrapper(y):
        return x + y
    return wrapper

adder5 = adder(5)
# 输出 15
adder5(10)
# 输出 11
adder5(6)

11

这比用类来实现更优雅，此外**装饰器也是基于闭包的一中应用场景**。

所有函数都有一个 `__closure__`属性，如果这个函数是一个闭包的话，那么它返回的是一个由 cell 对象 组成的元组对象。cell 对象的cell_contents 属性就是闭包中的自由变量。

In [6]:
adder.__closure__

In [7]:
adder5.__closure__

(<cell at 0x000001EF13228288: int object at 0x000000006386B160>,)

In [8]:
adder5.__closure__[0].cell_contents

5

这解释了为什么局部变量脱离函数之后，还可以在函数之外被访问的原因的，因为它存储在了闭包的 cell_contents中了。

# 用类实现

In [9]:
class Adder:
    def __init__(self,x):
        self.x = x
    
    def __call__(self,y):
        return self.x + y


In [10]:
add5 = Adder(5)

In [11]:
add5(6)

11

In [12]:
add5(10)

15

# 更进一步，基于类的装饰器

In [20]:
class Counter:
    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
#         print('call is run')
        return self.func(*args, **kwargs)

@Counter
def foo():
    pass

for i in range(10):
    foo()

print(foo.count)  # 10

call is run
call is run
call is run
call is run
call is run
call is run
call is run
call is run
call is run
call is run
10


首先这里的@Counter是装饰器，执行起来顺序是 foo = Counter(foo)， 这时foo已经是Counter实例，而不是本身foo函数。
当执行foo()的时候，其实已经变成了，执行__call__函数，而这个函数里面是执行了本身的self.func 即foo的实际逻辑， 而且加上了计算调用次数。这样就记录状态了。

In [17]:
foo.func()

foo is running
